Need to run this on a docker image with pyvista etc installed, e.g. `quay.io/jbusecke/pangeo_pyvista_docker_image:698eff3ebb26`

In [1]:
# !pip install -e ../../

In [2]:
from pyvista_tools.movie import Movie
from pyvista_tools.path import camera_path

import xarray as xr
import numpy as np
ds = xr.open_zarr("gs://leap-persistent/data-library/feedstocks/eNATL_feedstock/eNATL60-BLBT02.zarr", chunks={})
ds = ds.rename({'nav_lon':'lon', 'nav_lat':'lat'})

# mask out the land
mask = (ds['vosaline']>0).isel(time=0).reset_coords(drop=True)
ds = ds.where(mask)

# use a very short ds for testing
ds = ds.isel(time=slice(0,10))

In [3]:
# lets for now just keep the same position and move in closer
nframes = len(ds.time)
nmove = nframes//2
lon = np.linspace(-35, -60, nframes)
lat = np.linspace(30, 38, nframes)
# zoom in then stand still
r = np.linspace(2.8, 1.4, nframes)
path = camera_path(lon, lat, r)

In [4]:
import geovista as gv
gv.cache.blue_marble()

Texture (0x7dc96be99780)
  Components:   3
  Cube Map:     False
  Dimensions:   5400, 2700

In [5]:
factor = 1
ds_movie = ds.isel(x=slice(0,None, factor), y=slice(0,None, factor))

In [6]:
kwargs = {
    'vosaline':{'clim':[32.5, 38], 'cmap':'inferno'},
    'votemper':{'clim':[2, 30], 'cmap':'viridis'},
    'vovecrtz':{'clim':[-0.0005, 0.0005], 'cmap':'RdBu_r'},
}

In [7]:
var = 'vovecrtz'
da_preloaded = ds_movie[var].load() 

In [8]:
%%time
%%prun
m = Movie(
    da_preloaded,
    camera_path=path,
    **kwargs[var]
)
m.render(f'profiling.mp4', resolution=[1920, 1088])
# m.preview(0)

PolyData (0x7dc9684524a0)
  N Cells:    39492984
  N Points:   39506066
  N Strips:   0
  X Bounds:   -1.223e-01, 9.755e-01
  Y Bounds:   -9.778e-01, 4.964e-01
  Z Bounds:   1.107e-01, 9.231e-01
  N Arrays:   2
(39506066,)


  0%|          | 0/10 [00:00<?, ?it/s]

 CPU times: user 33min 17s, sys: 52.1 s, total: 34min 9s
Wall time: 8min 32s


         195450 function calls (190999 primitive calls) in 512.141 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       29  502.507   17.328  502.507   17.328 {method 'Render' of 'vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderWindow' objects}
        2    1.288    0.644    1.494    0.747 common.py:617(to_cartesian)
       10    1.099    0.110    1.099    0.110 {method 'ResetCameraClippingRange' of 'vtkmodules.vtkRenderingCore.vtkRenderer' objects}
        7    1.015    0.145    1.335    0.191 numeric.py:2249(isclose)
        1    0.929    0.929    0.929    0.929 {method 'SetCells' of 'vtkmodules.vtkCommonDataModel.vtkCellArray' objects}
       12    0.857    0.071    0.857    0.071 {method 'flatten' of 'numpy.ndarray' objects}
        2    0.770    0.385    0.770    0.385 shape_base.py:292(hstack)
        1    0.525    0.525    0.525    0.525 {method 'Finalize' of 'vtkmodules.vtkRenderingOpenGL2.vtkOSOpenGLRenderWindow' o

# Hmmm this is weird, even if I choose a 64 core machine I get very similar CPU % patterns (never exceeding 16) and time to run. I wonder if there is an issue with using cores across sockets?

See comment above. I need to understand why the cpu useage is generally so low? Might be that there is not enough work 

Some further improvements: 
- Thresholding outside of the loop?
- Use [ghosting](https://docs.pyvista.org/examples/02-plot/ghost-cells.html) instead of thresholding?
    - ! Actually I need neither! This works just fine without
- These had some improvement at lower res, but maybe the way to go is to use the data as textures, with a lower res mesh! https://banesullivan.com/pyvista/examples/geological-map.html
    - Getting more and more confident that this is the way unless we want to do something fancy like sea surface height as actual elevation with some other field as texture or something.

In [9]:
p = gv.GeoPlotter(window_size=None)

In [10]:
p.window_size

[1024, 768]

In [11]:
m

Movie(da=<xarray.DataArray 'vovecrtz' (x: 8354, y: 4729, time: 10)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, n

In [12]:
# m.preview(9)